In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
pd.options.display.max_rows = 100
from rich import print
import warnings
import datetime
from astropy.time import Time
import os
import astropy.units as u
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
import astropy.coordinates as coord
import astropy.units as u
import time
from gfcat_utils import parse_lightcurves_csv,parse_exposure_time,read_image
from matplotlib import gridspec
from astropy.visualization import ZScaleInterval
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
from scipy.interpolate import interp1d

In [2]:
def angularSeparation(ra1, dec1, ra2, dec2):
    d2r = np.pi/180.
    ra2deg = 1./d2r
    d1 = dec1*d2r,
    d2 = dec2*d2r
    r1 = ra1*d2r
    r2 = ra2*d2r
    a = np.sin((d2-d1)/2.)**2.+np.cos(d1)*np.cos(d2)*np.sin((r2-r1)/2.)**2.
    r = 2*np.arcsin(np.sqrt(a))
    return r*ra2deg

def counts2flux(cps, band):
    scale = 1.4e-15 if band == 'FUV' else 2.06e-16
    return scale*cps

def counts2mag(cps, band):
    scale = 18.82 if band == 'FUV' else 20.08
    # This threw a warning if the countrate was negative which happens when
    #  the background is brighter than the source. Suppress.
    with np.errstate(invalid='ignore'):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mag = -2.5 * np.log10(cps) + scale
    return mag

def mag2counts(mag, band):
    scale = 18.82 if band == 'FUV' else 20.08
    return 10.**(-(mag-scale)/2.5)

def apcorrect1(radius, band):
    if not band in ['NUV', 'FUV']:
        print("Invalid band.")
        return
    aper = np.array([1.5, 2.3, 3.8, 6.0, 9.0, 12.8, 17.3, 30., 60., 90.])/3600.
    if radius > aper[-1]:
        return 0.
    if band == 'FUV':
        dmag = [1.65, 0.96, 0.36, 0.15, 0.1, 0.09, 0.07, 0.06, 0.03, 0.01]
    else:
        dmag = [2.09, 1.33, 0.59, 0.23, 0.13, 0.09, 0.07, 0.04, -0.00, -0.01]
        if radius > aper[-2]:
            return 0.
    if radius < aper[0]:
        return dmag[0]
    ix = np.where((aper-radius) >= 0.)
    x = [aper[ix[0][0]-1], aper[ix[0][0]]]
    y = [dmag[ix[0][0]-1], dmag[ix[0][0]]]
    m, C = np.polyfit(x, y, 1)
    return m*radius+C

In [3]:
tbl = pd.read_csv('gfcat_visit_table_crossmatched.csv',index_col=None)
print(tbl.keys())
tbl

Index(['eclipse', 'obj_id', 'ra', 'dec', 'morphology', 'simbad_n_match',
       'simbad_otype', 'simbad_main_id', 'simbad_distance',
       'simbad_match_offset', 'obstart', 'datetime_iso', 'datetime_decimal',
       'gaia_source_id', 'gaia_distance_uncorrected', 'gaia_distance', 'pmra',
       'pmdec', 'gaia_ra', 'gaia_dec', 'gaia_match_offset',
       'gaia_match_offset_uncorrected', 'gaia_n_match', 'distance'],
      dtype='object')

,eclipse,obj_id,ra,dec,morphology,simbad_n_match,simbad_otype,simbad_main_id,simbad_distance,simbad_match_offset,...,gaia_distance_uncorrected,gaia_distance,pmra,pmdec,gaia_ra,gaia_dec,gaia_match_offset,gaia_match_offset_uncorrected,gaia_n_match,distance
0,26808,20726808,185.750875,29.544538,T,1,Star,BD+30 2263,688.2312456985546,2.5232010327743186,...,688.2149791694012,688.2149791694012,-27.79079965740919,20.885018996980794,185.75020343128907,29.545159192855937,3.0693762291788986,3.0693762291788986,1,688.231246
1,21378,39121378,204.877337,30.654018,U,1,WhiteDwarf_Candidate,SDSS J133930.42+303915.8,207.91317545792876,2.40529994681043,...,207.91219849037992,207.91219849037992,-21.451225639490513,-77.25055300915739,204.87665763054886,30.654129477810436,2.142084424655653,2.142084424655653,1,207.912198
2,26045,41026045,149.468637,2.338955,U,14,"AGN, Galaxy, GroupG, QSO",[VV2006] J095752.3+022021,17667.8445229682,2.5117083182712765,...,320.98005380703074,320.98005380703074,29.687377507525213,-25.398667661725018,149.45969039599856,2.379447056818682,149.28248831248132,149.28248831248132,0,17667.844523
3,23102,42723102,257.253864,58.873073,F,0,--,--,--,--,...,535.0802765374402,535.0802765374402,-12.129245356465246,-31.545547494242435,257.2489767188669,58.858514370552946,53.1936273510468,53.1936273510468,0,535.080277
4,17070,44617070,247.108147,40.861124,F,2,"EmissionG, GroupG, Star",[KB98] Obj J162826+405143,54.69143094659928,2.408692784050565,...,54.69134496235111,54.69134496235111,-26.558492748544055,-8.000748311927003,247.10741335743467,40.861594911788295,2.6207323245198824,2.6207323245198824,1,54.691431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1524,43201,99999943201,217.158560,57.753320,U,1,Star,HD 238354,293.470286133529,1.3910082890863427,...,293.4737191702322,293.4737191702322,4.0804689349688275,-24.134876923294346,217.15859674939682,57.752826538050414,1.7778649583038464,1.7778649583038464,1,293.470286
1525,43238,99999943238,243.411230,54.665030,F,0,--,--,--,--,...,75.56206085149886,75.56206085149886,-55.580392743412716,67.51099969033856,243.4104097632804,54.664671086272,2.1415175225418004,2.1415175225418004,1,75.562061
1526,43828,99999943828,338.340990,-60.930260,F,0,--,--,--,--,...,92.94354505094304,92.94354505094304,63.22009632448086,-22.30958384842886,338.3411479532575,-60.930596253363916,1.2416405335793734,1.2416405335793734,1,92.943545
1527,44684,99999944684,308.132040,7.971320,F,0,AGN_Candidate,FIRST J203232.8+075805,--,21.377871377979307,...,163.76240587886684,163.76240587886684,-15.632556922872418,-9.6567354412478,308.13165525685173,7.970970492133854,1.8613649789408448,1.8613649789408448,3,163.762406


In [4]:
# e37477 has two variables, so use that as a tester

In [5]:
for i in np.arange(len(tbl)):
    visit = tbl.iloc[i]
    e = visit['eclipse']
    edir = str(e).zfill(5)
    epath = f"../data/lightcurves/e{edir}"
    ra,dec = visit['ra'],visit['dec']
    obs = {}
    for band in ['NUV','FUV']:
        fn = f"{epath}/e{edir}-{band[0].lower()}d-30s-photom-17_5.csv"
        try:
            expt = parse_exposure_time(f"{epath}/e{edir}-{band[0].lower()}d-30s-exptime.csv")
            lcs = parse_lightcurves_csv(fn)
            # a few eclipse have multiple variables, so we have to pick the right one
            ix = np.argmin(angularSeparation(ra,dec,
                           np.array([lc['ra'] for lc in lcs]),
                           np.array([lc['dec'] for lc in lcs])))
            lc = lcs[ix]
        except FileNotFoundError:
            continue
        obs[band] = [lc,expt]

        # add observation minimum flux
        ix = np.where(lc['cps']==np.nanmin(lc['cps'][np.where(lc['cps']!=0)][:-1]))
        tbl.loc[i,f'cps_max_{band}']=lc['cps'][ix]
        tbl.loc[i,f'cps_max_err_{band}']=lc['cps_err'][ix]
        tbl.loc[i,f'mag_max_{band}']=counts2mag(lc['cps'][ix],band)
        tbl.loc[i,f'mag_max_err_{band}']=np.abs(
            counts2mag(lc['cps_err'][ix]+lc['cps'][ix],band)-counts2mag(lc['cps'][ix],band))
        
        # observation maximum flux
        ix = np.where(lc['cps']==np.nanmax(lc['cps'][np.where(np.isfinite(lc['cps']))][:-1]))
        tbl.loc[i,f'cps_min_{band}']=lc['cps'][ix]
        tbl.loc[i,f'cps_min_err_{band}']=lc['cps_err'][ix]
        tbl.loc[i,f'mag_min_{band}']=counts2mag(lc['cps'][ix],band)
        tbl.loc[i,f'mag_min_err_{band}']=np.abs(
            counts2mag(lc['cps_err'][ix]+lc['cps'][ix],band)-counts2mag(lc['cps'][ix],band))

        # observation mean flux
        t = np.array(expt['expt_eff'])[np.where(np.isfinite(lc['cps']) & (lc['cps']!=0))][:-1]
        c = lc['cps'][np.where(np.isfinite(lc['cps']) & (lc['cps']!=0))][:-1]
        tbl.loc[i,f'cps_mean_{band}']=(c*t).sum()/t.sum()
        tbl.loc[i,f'cps_mean_err_{band}']=np.sqrt((c*t).sum())/t.sum()
        tbl.loc[i,f'mag_mean_{band}']=counts2mag((c*t).sum()/t.sum(),band)
        tbl.loc[i,f'mag_mean_err_{band}']=np.abs(
            counts2mag(np.sqrt((c*t).sum())/t.sum()+(c*t).sum()/t.sum(),band)-counts2mag((c*t).sum()/t.sum(),band))

        
        # add valid data start time, stop time, duration
        ix = np.where(np.isfinite(lc['cps']))
        tbl.loc[i,f'tstart_{band}'] = np.round(np.min(np.array(expt['t0'])[ix]))
        tbl.loc[i,f'tstop_{band}'] = np.round(np.max(np.array(expt['t0'])[ix]))
        tbl.loc[i,f'expt_raw_{band}'] = (np.round(np.max(np.array(expt['t0'])[ix])) -
                                        np.round(np.min(np.array(expt['t0'])[ix])))

    # if there is data for both bands, add the color information
    if len(obs.keys())==2:
        _,ix,_ = np.intersect1d(obs['NUV'][1]['t0'],obs['FUV'][1]['t0'],return_indices=True)
        color = (counts2flux(obs['FUV'][0]['cps'],'FUV')[ix][:-1] /
                 counts2flux(obs['NUV'][0]['cps'],'NUV')[ix][:-1])
        color_err = color * np.sqrt(
            (counts2flux(obs['NUV'][0]['cps_err'],'NUV')[ix][:-1] /
             counts2flux(obs['NUV'][0]['cps'],    'NUV')[ix][:-1])**2 +
            (counts2flux(obs['FUV'][0]['cps_err'],'FUV')[ix][:-1] /
             counts2flux(obs['FUV'][0]['cps'],    'FUV')[ix][:-1])**2)
        expt = obs['FUV'][1]
        ix = np.where(color==np.nanmin(color[np.where(color!=0)]))
        tbl.loc[i,'ratio_min'] = color[ix]
        tbl.loc[i,'ratio_min_err'] = color_err[ix]
        ix = np.where(color==np.nanmax(color[np.where(np.isfinite(color))]))
        tbl.loc[i,'ratio_max'] = color[ix]
        tbl.loc[i,'ratio_max_err'] = color_err[ix]
        ix = np.where(lc['cps']==np.nanmax(lc['cps'][np.where(np.isfinite(color))]))
        tbl.loc[i,'ratio_at_nuv_max'] = color[ix]
        tbl.loc[i,'ratio_at_nuv_max_err'] = color_err[ix]

In [6]:
apers = [9,12.5,17.5,25,35]
for i in np.arange(len(tbl)):
    visit = tbl.iloc[i]
    e = visit['eclipse']
    edir = str(e).zfill(5)
    epath = f"../data/lightcurves/e{edir}"
    ra,dec = visit['ra'],visit['dec']
    for band in ['NUV','FUV']:
        try:
            expt = parse_exposure_time(f"{epath}/e{edir}-{band[0].lower()}d-30s-exptime.csv")
        except FileNotFoundError: # no data for this band
            continue
        tbl.loc[i,f'expt_eff_{band}'] = sum(expt['expt_eff'])
        for aper in apers:
            aperstr = str(aper).replace('.','_')
            fn = f"{epath}/e{edir}-{band[0].lower()}d-30s-photom-{aperstr}.csv"
            try:
                lcs = parse_lightcurves_csv(fn)
                # a few eclipse have multiple variables, so we have to pick the right one
                ix = np.argmin(angularSeparation(ra,dec,
                               np.array([lc['ra'] for lc in lcs]),
                               np.array([lc['dec'] for lc in lcs])))
                lc = lcs[ix]
            except FileNotFoundError: # no data for this aperture? but data for this band. shouldn't happen.
                print(f"Error finding {fn}")
                continue
            tbl.loc[i,f'counts_{aperstr}'] = np.sum(lc['counts'])
        aper_area = np.pi*17.5**2
        annulus_area = np.pi*35**2-np.pi*25**2
        tbl.loc[i,f'skybg_counts_{band}']=(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25'])*aper_area/annulus_area
        tbl.loc[i,f'skybg_counts_err_{band}']=np.sqrt(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25'])*aper_area/annulus_area
        tbl.loc[i,f'skybg_flux_{band}']=counts2flux(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25'],band)*aper_area/annulus_area
        tbl.loc[i,f'skybg_flux_err_{band}']=counts2flux(np.sqrt(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25']*aper_area/annulus_area),band)

In [15]:
tbl.replace(np.nan, '--', inplace=True)
tbl.sort_values(['ra'],inplace=True)
tbl.to_csv('gfcat_visit_table_lcstats.csv',index=None)
tbl

,eclipse,obj_id,ra,dec,morphology,simbad_n_match,simbad_otype,simbad_main_id,simbad_distance,simbad_match_offset,...,counts_35,skybg_counts_NUV,skybg_counts_err_NUV,skybg_flux_NUV,skybg_flux_err_NUV,expt_eff_FUV,skybg_counts_FUV,skybg_counts_err_FUV,skybg_flux_FUV,skybg_flux_err_FUV
1216,34085,1628034085,1.012610,-4.692878,T,1,Variable*,ATO J001.0125-04.6926,2558.8536335721597,1.0912917153506068,...,20735.364755,1927.191086,31.363521,0.0,0.0,--,--,--,--,--
1072,34227,1337334227,2.248695,23.643045,T,1,**,ADS 106 AB,--,1.1062231150441744,...,261858.266497,5347.262066,52.243006,0.0,0.0,--,--,--,--,--
497,28485,662128485,2.427261,-0.563277,T,1,RRLyrae,ATO J002.4268-00.5629,25380.71065989848,1.9210325389059826,...,779.625709,1813.736469,30.426326,0.0,0.0,1463.819437,179.480941,9.571315,0.0,0.0
909,28486,1114928486,2.706065,-0.648669,F,0,--,--,--,--,...,832.888346,1624.103146,28.791827,0.0,0.0,1466.928573,188.690364,9.813802,0.0,0.0
891,13770,1088713770,4.049087,-39.456450,T,1,RRLyrae,CRTS J001611.7-392722,5546.311702717693,1.9673644712518494,...,992.075077,1706.75991,29.515398,0.0,0.0,1475.304485,170.469521,9.327941,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,28520,259328520,359.085945,-4.492257,U,1,"HighPM*, X",SCR J2356-0429,25.518145953587595,3.2850030069962233,...,552.01419,1419.017506,26.91264,0.0,0.0,1407.177502,115.042161,7.662861,0.0,0.0
608,44465,786144465,359.256526,-55.897952,F,0,--,--,--,--,...,5447.328375,1319.379419,25.950592,0.0,0.0,--,--,--,--,--
434,28561,587528561,359.336534,-12.980567,F,1,"**, Eruptive*",LP 704-15,19.831944105648734,2.9998324595644084,...,827.840623,1626.662597,28.814505,0.0,0.0,1604.555159,117.814459,7.754641,0.0,0.0
1164,28525,1509128525,359.491560,-3.689567,F,1,EclBin_Candidate,KIC 60019244,321.75032175032175,1.8061054819406248,...,950.543368,1828.926422,30.55347,0.0,0.0,1550.718663,187.60499,9.785536,0.0,0.0


In [8]:
apers = [9,12.5,17.5,25,35]
print("Estimated aperture corrections on a")
for aper in apers:
    print(f"  =>  {aper}\" aperture is {np.round(100*apcorrect1(aper/60/60,'NUV'),1)}% in NUV and {np.round(100*apcorrect1(aper/60/60,'FUV'),1)}% in FUV")


Estimated aperture corrections on a

=>  9" aperture is 13.0% in NUV and 10.0% in FUV

=>  12.5" aperture is 9.3% in NUV and 9.1% in FUV

=>  17.5" aperture is 7.0% in NUV and 7.0% in FUV

=>  25" aperture is 5.2% in NUV and 6.4% in FUV

=>  35" aperture is 3.3% in NUV and 5.5% in FUV